# 2. Data Cleaning
**Group Project for DATA INFORMATION AND QUALITY (2024-2025) <br>**
Analysis of Milan Personal Services - Database 12 <br>
Mauro Orazio Drago, Dennis Pierantozzi, Davide Morelli

The project focuses on performing *data cleaning* tasks on the dirty dataset to create a final cleaned dataset.

### Import the repository from GitHub

First of all we start by importing the repository that we stored in the github project.

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("NEW_GITHUB_TOKEN")

In [2]:
token = UserSecretsClient().get_secret("NEW_GITHUB_TOKEN")
! git clone https://{token}@github.com/madratak/Milan_Services_2012.git

Cloning into 'Milan_Services_2012'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 185 (delta 0), reused 0 (delta 0), pack-reused 183 (from 1)
Receiving objects: 100% (185/185), 4.97 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (82/82), done.


### Set up the dataset

At this point we can proceed by importing the correct libraries and then importing the data itself inside our notebook.

In [3]:
# to compute the edit distance
!pip install rapidfuzz 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re
from rapidfuzz import fuzz
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import jaccard

# Set max column width to None to display full content
pd.set_option('display.max_colwidth', None)

In [5]:
%cd Milan_Services_2012

/kaggle/working/Milan_Services_2012


In [6]:
SERVICES = pd.read_csv('data/raw/Comune-di-Milano-Servizi-alla-persona-parrucchieri-estetisti.csv',sep=';',encoding='unicode_escape')
SERVICES["Codice via"] = SERVICES["Codice via"].fillna("").astype(str)
SERVICES.head()

,Tipo esercizio pa,Ubicazione,Tipo via,Via,Civico,Codice via,ZD,Prevalente,Superficie altri usi,Superficie lavorativa
0,NaN,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,NaN,55.0
1,NaN,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,CENTRO MASSAGGI RILASSANTI NON ESTETICI,2.0,28.0
2,NaN,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,CENTRO BENESSERE,2.0,27.0
3,NaN,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,TRUCCO SEMIPERMANENTE,NaN,NaN
4,NaN,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,NaN,50.0


### Import external dataset
During the data cleaning stage, external are used to fix irregular values, among these we have

In [7]:
viario = pd.read_csv('data/external/VIARIO_20241104.csv',sep=';',encoding='unicode_escape')
viario = viario.astype(str)

centroidi = pd.read_csv('data/external/CENTROIDI_20241104.csv',sep=';',encoding='unicode_escape')
centroidi = centroidi.astype(str)


tipovia = pd.read_csv('data/external/TIPOVIA.csv',sep=';',encoding='unicode_escape')
viario_centroidi = viario.merge(centroidi, on='CODICE_VIA', how='left')

civici = pd.read_csv('data/external/CiviciCdMI.csv',sep=';',encoding='unicode_escape')
civici = civici.astype(str)

## Wrangling 

This script focuses on renaming the columns of the `SERVICES` dataset to shorter, more convenient names.  

The original DataFrame is modified in place, and a copy of the updated dataset, named `new_SERVICES`, is created for further analysis or processing.  

We pass from:

| Tipo esercizio pa     | Ubicazione     | Tipo via | Via      | Civico | Codice via | ZD | Prevalente | Superficie altri usi | Superficie lavorativa |
|-------|-------|------|-------|-----|-----|---------|----|---------|---------|

to:


| t_es  | ubic | t_via | via | civ | cod_via | zd | main | sup_alt | sup_lav | 
|-------|------|-------|-----|-----|---------|----|------|---------|---------|


In [8]:
new_SERVICES = SERVICES.copy()

new_SERVICES = new_SERVICES.rename(columns={
    'Tipo esercizio pa': 't_es',
    'Ubicazione': 'ubic',
    'Tipo via': 't_via',
    'Via': 'via',
    'Civico': 'civ',
    'Codice via': 'cod_via',
    'ZD': 'zd',
    'Prevalente': 'main',
    'Superficie altri usi': 'sup_alt',
    'Superficie lavorativa': 'sup_lav'
})

new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,main,sup_alt,sup_lav
0,NaN,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,NaN,55.0
1,NaN,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,CENTRO MASSAGGI RILASSANTI NON ESTETICI,2.0,28.0
2,NaN,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,CENTRO BENESSERE,2.0,27.0
3,NaN,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,TRUCCO SEMIPERMANENTE,NaN,NaN
4,NaN,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,NaN,50.0


## Transformation
### ZD: Removing data representing different information

This script processes the `zd` column in the `SERVICES` dataset. The column initially contains a mix of numeric values, missing values (`nan`), and a single non-numeric label `'ACCONCIATORE'`.  

**Goal**: to filter out or modify rows where `zd` contains irrelevant or inconsistent information (e.g., `'ACCONCIATORE'`) to ensure that only meaningful numeric data is retained for analysis.

In [9]:
new_SERVICES.zd.unique()

array(['6', '9', '5', '1', '4', '2', '8', '3', nan, 'ACCONCIATORE', '7'],
      dtype=object)

During the data cleaning process, we identified two consecutive rows in the `SERVICES` dataset that appear to represent the same entity but contain fragmented or overlapping information. Specifically:

1. The first row (index 32) contains descriptive information about the entity, such as `'Acconciatore'` in the `t_es` column, but lacks structured data for most other fields.
2. The second row (index 33) provides structured data for fields like `via`, `civ`, `cod_via`, and `zd`, but the `t_es` column contains a reference to `'ACCONCIATORE'`.

In [10]:
new_SERVICES.loc[32:33].head()

,t_es,ubic,t_via,via,civ,cod_via,zd,main,sup_alt,sup_lav
32,Acconciatore,CSO COMO N. 15 interno club f. conti,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
33,(z.d. 9),CSO,COMO,15,1111,9.0,ACCONCIATORE,NaN,195.0,NaN


To standardize and improve the dataset:
- The two rows were merged into a single entry, combining the fragmented information into a complete record. 
- This process ensures no information is lost while achieving the desired structure and format for the full dataset.

##### Example:
Before merging:
- Row 32: `'Acconciatore', 'CSO COMO N. 15 interno club f. conti', NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN`
- Row 33: `'(z.d. 9)', 'CSO', 'COMO', '15', '1111', '9.0', 'ACCONCIATORE', NaN, '195.0', NaN`

After merging:
- New Row: `'Acconciatore', 'CSO COMO', '15', '1111', '9.0', NaN, NaN, '195.0', Nan`

In [11]:
# New merged row
merged_row = {
    "t_es": "Acconciatore",
    "ubic": "CSO COMO N. 15 (z.d. 9)",
    "t_via": "CSO",
    "via": "CSO COMO",
    "civ": "15",
    "cod_via": 1111.0,
    "zd": "9",
    "main": "ACCONCIATORE",
    "sup_alt": np.nan,
    "sup_lav": 195.0,
}

# Remove row at index 33 and replace row at index 32
new_SERVICES = new_SERVICES.drop(index=33, errors='ignore')  # Drop row 33
new_SERVICES.loc[32] = merged_row  # Add the merged row at index 32

# Ensure correct row order and reset index if necessary
new_SERVICES = new_SERVICES.sort_index().reset_index(drop=True)

new_SERVICES.loc[32:33].head()

,t_es,ubic,t_via,via,civ,cod_via,zd,main,sup_alt,sup_lav
32,Acconciatore,CSO COMO N. 15 (z.d. 9),CSO,CSO COMO,15,1111.0,9,ACCONCIATORE,NaN,195.0
33,Acconciatore,CSO DI PORTA NUOVA N. 46 ; (z.d. 1),CSO,DI PORTA NUOVA,46,1044.0,1,NaN,NaN,34.0


Finally, the available values in the column `zd` are:

In [12]:
np.sort(new_SERVICES['zd'].unique())

array(['1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)

### T_ES: Label reduction

The `t_es` column in the dataset contains a variety of labels that represent different categories of services. Many of these labels have similar meanings or represent variations of the same concept, leading to redundancy and potential inconsistencies in analysis.

To address this, we will reduce the number of labels by grouping similar ones under a unified label, making the dataset easier to interpret and analyze while preserving the essential distinctions between categories.

In [13]:
print("The number of different label combination for 't_es' column is {}.".format(new_SERVICES.t_es.unique().shape[0]))

The number of different label combination for 't_es' column is 103.


In [14]:
# Ensure 't_es' column does not have NaN values and is treated as strings
new_SERVICES["t_es"] = new_SERVICES["t_es"].fillna("").astype(str)

# Split the 't_es' column by ';' and flatten the resulting lists
flattened_labels = [label.strip() for item in new_SERVICES["t_es"] for label in item.split(";")]

# Get the unique labels and sort them
unique_labels = sorted(set(flattened_labels))

# Initialize a dictionary to store counts
label_counts = {label: 0 for label in unique_labels}

# Loop through each row in the "t_es" column and count the occurrences of each label
for entry in new_SERVICES['t_es'].dropna():  # Drop NaN values to avoid errors
    for label in unique_labels:
        if label in entry:
            label_counts[label] += 1

print("COUNTS of unique labels\n")

# Find the maximum width for labels and numbers
max_label_length = max(len(label) for label in label_counts if label != "")
max_number_length = max(len(str(count)) for count in label_counts.values())

# Print the counts with aligned output
for label, count in label_counts.items():
    if label != "":
        print(f"{label:<{max_label_length}} \t{count:>{max_number_length}}")

COUNTS of unique labels

ACCONCIATORE                      	 717
Acconciatore                      	 190
BARBIERE                          	   2
Centro abbronzatura               	   2
Centro benessere                  	  29
Centro massaggi                   	 139
Esecuzione di tatuaggi e piercing 	   5
Estetista                         	  31
Estetista in profumeria           	   2
Manicure                          	   3
Parrucchiere misto                	 105
Parrucchiere per signora          	1213
Parrucchiere per uomo             	 518
Pedicure estetico                 	  22
TIPO A - REG.2003                 	 865
TIPO A ESTETICA MANUALE           	 263
TIPO A-B-C-D                      	  88
TIPO B CENTRO DI ABBRONZATURA     	 603
TIPO C TRATT.ESTETICI DIMAGRIM    	 138
TIPO D ESTET.APPAR.ELETTROMECC    	  74
Truccatore                        	   1
esecuzione di tatuaggi e piercing 	  19


Now we transform and standardize the values in the `t_es` column of the dataset. It consists of two main steps:

1. **Composite Labels Handling**: Some rows contain multiple related labels that should be split or expanded into individual categories. For example, labels like `"TIPO A-B-C-D"` or `"Parrucchiere misto"` are expanded into separate labels to ensure all relevant categories are included.
   
2. **Label Mapping**: After handling composite labels (e.g., splitting `"TIPO A-B-C-D"` into individual components), the code replaces each original label (such as `"Centro abbronzatura"`) with the most similar label from a predefined set using the **Jaccard** similarity measure. This step ensures that similar labels are grouped under a single, consistent label based on the highest similarity score.

##### Example:

Before transformation:
- `"TIPO A-B-C-D; Parrucchiere misto"`

After transformation:
- `"Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura; Tipo C - Trattamenti Estetici Dimagrimento; Tipo D - Estetica Apparati Elettromeccanici; Parrucchiere per Uomo; Parrucchiere per Donna"`

This process ensures that the dataset contains consistent and meaningful labels, making the data easier to analyze and reducing redundancy.

In [15]:
target_category_dict = {
    'Acconciatore': ['Acconciatore'],
    'Parrucchiere per Uomo': ['Barbiere', 'Parrucchiere per Uomo'],
    'Tipo B - Centro di Abbronzatura': ['Tipo B - Centro di Abbronzatura'],
    'Centro Benessere': ['Centro benessere'],
    'Centro Massaggi': ['Centro massaggi'],
    'Esecuzione di Tatuaggi e Piercing': ['Esecuzione di tatuaggi e piercing'],
    'Tipo A - Estetica Manuale': ['Estetista in profumeria', 'Manicure', 'Pedicure estetico', 'Tipo A - Estetica Manuale', 'TIPO A - REG.2003'],
    'Parrucchiere per Donna': ['Parrucchiere per signora'],
    'Truccatore': ['Truccatore'],
    'Tipo C - Trattamenti Estetici Dimagrimento': ['Tipo C - Trattamenti Estetici Dimagrimento'],
    'Tipo D - Estetica Apparati Elettromeccanici': ['Tipo D - Estetica Apparati Elettromeccanici']
}

# Function to handle composite labels (if applicable)
def handle_composite_labels(value):
    # If composite labels exist (e.g., "TIPO A-B-C-D", "Parrucchiere misto"), expand them
    if "TIPO A-B-C-D" in value:
        value = value.replace("TIPO A-B-C-D", "; ".join([
            "Tipo A - Estetica Manuale", 
            "Tipo B - Centro di Abbronzatura",
            "Tipo C - Trattamenti Estetici Dimagrimento", 
            "Tipo D - Estetica Apparati Elettromeccanici"
        ]))
    if "Parrucchiere misto" in value:
        value = value.replace("Parrucchiere misto", "; ".join([
            "Parrucchiere per Uomo",
            "Parrucchiere per Donna"
        ]))
    return value

# Jaccard similarity function
def jaccard_similarity(str1, str2):
    set1 = set(str1.lower().split())
    set2 = set(str2.lower().split())
    return len(set1 & set2) / len(set1 | set2)

# Define the function to map a label to the best matching category using Jaccard similarity
def map_label_to_category_jaccard(label, target_category_dict):
    best_match = None
    best_score = 0
    
    for target, variations in target_category_dict.items():
        for variation in variations:
            score = jaccard_similarity(label, variation)  # Calculate Jaccard similarity
            if score > best_score:
                best_match = target
                best_score = score
                
    return best_match

# Apply the mapping for each label in the "t_es" column using Jaccard similarity
def safe_apply_mapping(x, target_category_dict):
    if isinstance(x, str) and x:  # Check if x is a non-empty string
        return "; ".join(
            [map_label_to_category_jaccard(label, target_category_dict) for label in x.split(';')]
        )
    return ""  # If the value is None, NaN, or empty string, return an empty string

# Apply the safe function to the "t_es" column
new_SERVICES["t_es"] = new_SERVICES["t_es"].apply(lambda x: safe_apply_mapping(x, target_category_dict))

# Show the updated DataFrame
new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,main,sup_alt,sup_lav
0,,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,NaN,55.0
1,,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,CENTRO MASSAGGI RILASSANTI NON ESTETICI,2.0,28.0
2,,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,CENTRO BENESSERE,2.0,27.0
3,,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,TRUCCO SEMIPERMANENTE,NaN,NaN
4,,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,NaN,50.0


In [16]:
# Split the 't_es' column by ';' and flatten the resulting lists
flattened_labels = [label.strip() for item in new_SERVICES["t_es"] for label in item.split(";")]

# Get the unique labels and sort them
unique_labels = sorted(set(flattened_labels))

# Initialize a dictionary to store counts
label_counts = {label: 0 for label in unique_labels}

# Loop through each row in the "t_es" column and count the occurrences of each label
for entry in new_SERVICES['t_es']:  # No need to drop NaN values as we've filled them earlier
    for label in unique_labels:
        if label in entry:
            label_counts[label] += 1

# Find the maximum width for labels and numbers
max_label_length = max(len(label) for label in label_counts if label.strip() != "")
max_number_length = max(len(str(count)) for count in label_counts.values())

print("COUNTS of new unique labels\n")
# Print the counts with aligned output
for label, count in label_counts.items():
    if label.strip() != "":  # Ensure labels are not empty
        print(f"{label:<{max_label_length}} \t{count:>{max_number_length}}")

COUNTS of new unique labels

Acconciatore                                	 907
Centro Benessere                            	  29
Centro Massaggi                             	 139
Esecuzione di Tatuaggi e Piercing           	  24
Parrucchiere per Donna                      	1213
Parrucchiere per Uomo                       	 601
Tipo A - Estetica Manuale                   	1266
Tipo B - Centro di Abbronzatura             	 605
Tipo C - Trattamenti Estetici Dimagrimento  	 138
Tipo D - Estetica Apparati Elettromeccanici 	  74
Truccatore                                  	   1


### MAIN: Label Reduction

In this step, we focus on standardizing the labels in the `main` column. Many of the labels in the column are variations of the same service but are written differently. To ensure consistency across the dataset, we map these different variations to a single, standardized label.

For example, variations like `"CENTRO MASSAGGI"` and `"CENTRO MASSAGGI RILASSANTI NON ESTETICI"` are all mapped to the single label `"Centro Massaggi"`. Similarly, terms like `"ESTETICA A E SOLARIUM"` are transformed into a combination of labels, such as `"Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura"`, to better reflect the service offerings.

This standardization ensures that similar services are grouped together under a consistent set of labels, reducing redundancy and making the dataset more uniform for further analysis.

**Example**:

- Before: `"CENTRO MASSAGGI RILASSANTI NON ESTETICI"` → After: `"Centro Massaggi"`
- Before: `"ESTETICA A E SOLARIUM"` → After: `"Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura"`

In [17]:
print("The number of different label combination  for column 'main' is {}.".format(new_SERVICES.main.unique().shape[0]))

The number of different label combination  for column 'main' is 64.


To do what we explained before, we create a `target_category_dict` dictionary where each key represents a main category (like `"Centro Massaggi"`), and the corresponding value is a list of related variations (such as `"Estetista"`, `"Manicure"`).<br>
This was developed after several failed attempts to map labels due to a lack of knowledge about variations, with the dictionary providing a reliable method to map labels to categories. <br>
The `map_label_to_category` function compares each label in the `main` column with the variations in the dictionary using **Jaccard** similarity, and returns the category with the highest similarity score. 


This approach standardizes the `main` column by mapping similar labels to consistent categories.

In [18]:
target_category_dict = {
    'Centro Massaggi': ['Non Estetici', 'Centro Massaggi'],
    'Esecuzione di Tatuaggi e Piercing': ['Esecuzione di Tatuaggi e Piercing', 'Tatuaggio', 'Piercing'],
    'Acconciatore': ['Acconciatore', 'Parrucchiere', 'Aconciatore'],
    'Parrucchiere per Donna': ['Donna', 'Uomo', 'Barbiere'],
    'Tipo A - Estetica Manuale': ['Tipo a - estetica manuale', 'Estetista', 'Estetica', 'Manicure', 'Pedicure', 'Unghie'],
    'Centro Benessere': ['Centro Benessere', 'Olistico', 'Benessero Fisico', 'Non Estetici'],
    'Truccatore': ['Trucco', 'Truccatore'],
    'Tipo B - Centro di Abbronzatura': ['Tipo B - Centro di Abbronzatura', 'Solarium']
}

# Function to map a label to the best matching category from the dictionary
def map_label_to_category(label, target_category_dict):
    if isinstance(label, str):  # Ensure label is a string
        best_match = None
        best_score = 0

        for target, variations in target_category_dict.items():
            for variation in variations:
                score = jaccard_similarity(label, variation)  # Calculate Jaccard similarity
                if score > best_score:
                    best_match = target
                    best_score = score

        return best_match
    else:
        return None  # Return None for non-string labels (NaN or float)

# Apply the mapping to the 'main' column in new_SERVICES
new_SERVICES['main'] = new_SERVICES['main'].apply(lambda x: map_label_to_category(x, target_category_dict))

In [19]:
# Initialize a dictionary to store counts
unique_main_labels = new_SERVICES["main"].dropna().unique()

label_counts = {label: 0 for label in unique_main_labels}

# Loop through each row in the "main" column and count the occurrences of each label
for entry in new_SERVICES['main'].dropna():  # Drop NaN values to avoid errors
    if isinstance(entry, str):  # Ensure the entry is a string
        for label in label_counts:
            if label in entry:
                label_counts[label] += 1

# Find the maximum width for labels and numbers
max_label_length = max(len(label) for label in label_counts if label != "")
max_number_length = max(len(str(count)) for count in label_counts.values())

print("COUNTS of new unique labels\n")
# Print the counts with aligned output
for label, count in label_counts.items():
    if label != "":
        print(f"{label:<{max_label_length}} \t{count:>{max_number_length}}")

COUNTS of new unique labels

Centro Massaggi                   	95
Centro Benessere                  	15
Truccatore                        	 1
Tipo A - Estetica Manuale         	61
Esecuzione di Tatuaggi e Piercing 	20
Acconciatore                      	98
Tipo B - Centro di Abbronzatura   	 4


Once standardized the column, we decide to move the values from the `main` column to the `t_es` column. 

The `main` column contains many null values, and the few non-null values it holds are not significantly different from the information already present in the `t_es` column. Therefore, we use the `main` column values to populate the `t_es` column where `t_es` is null, ensuring that we maintain all relevant information in a single column.

After this step, the `main` column will be dropped, as its contribution is minimal and redundant.

In [20]:
new_SERVICES['t_es'] = new_SERVICES['t_es'].replace("", float("nan"))
print("The initial number of Nan in 't_es' column is {}.".format(sum(new_SERVICES["t_es"].isnull())))

The initial number of Nan in 't_es' column is 31.


In [21]:
# Ensure both columns are strings and handle NaN values
new_SERVICES['t_es'] = new_SERVICES['t_es'].fillna("").astype(str)

def update_t_es(row):
    main_values = row['main'].split(";") if row['main'] else []
    t_es_values = row['t_es'].split(";") if row['t_es'] else []
    
    # Remove extra whitespace around the labels
    main_values = [value.strip() for value in main_values]
    t_es_values = [value.strip() for value in t_es_values]
    
    # If t_es is empty, copy main values
    if not t_es_values:
        return ";".join(main_values)
    
    # Add main labels that are not already in t_es
    for main_value in main_values:
        # Add the condition: If main is "Acconciatore" and t_es contains "Parrucchiere per Uomo" or "Parrucchiere per Donna", don't add
        if main_value == "Acconciatore" and any(x in t_es_values for x in ["Parrucchiere per Uomo", "Parrucchiere per Donna"]):
            continue
        if main_value not in t_es_values:
            t_es_values.append(main_value)
    
    # Return the updated t_es column as a semicolon-separated string
    return ";".join(t_es_values)

# Apply the function row-wise
new_SERVICES['t_es'] = new_SERVICES.apply(update_t_es, axis=1)

In [22]:
new_SERVICES['t_es'] = new_SERVICES['t_es'].replace("", float("nan"))
print("The number of Nan in 't_es' column afterwards is {}.".format(sum(new_SERVICES["t_es"].isnull())))
new_SERVICES['t_es'] = new_SERVICES['t_es'].fillna("").astype(str)

The number of Nan in 't_es' column afterwards is 16.


Since `main` info is incorporated in `t_es` column, we drop it.

In [23]:
new_SERVICES = new_SERVICES.drop('main', axis=1)

new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
0,,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,55.0
1,Centro Massaggi,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,2.0,28.0
2,Centro Benessere,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,2.0,27.0
3,Truccatore,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,NaN,NaN
4,,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,50.0


### T_VIA: Checking format
In this phase, we verify that the `t_via` column contains only valid and properly formatted values.

The column contains designations for the type of street or location, which are standardized according to the tipovia dataframe. This dataframe defines valid labels and their corresponding extended and abbreviated descriptions.

In [24]:
# Ensure comparison is case-insensitive by converting both columns to lowercase
t_via_unique = new_SERVICES['t_via'].unique()
desc_abbreviata = tipovia['DESC_ABBREVIATA'].str.lower().unique()

# Find t_via values not in tipovia.DESC_ABBREVIATA
invalid_t_via = [via for via in t_via_unique if via.lower() not in desc_abbreviata]

# Print the results
print("Invalid t_via values not in tipovia:\n")
print(invalid_t_via)


Invalid t_via values not in tipovia:

['VIE']


Since any potential unformatted or invalid values in the `t_via` column were already addressed during the previous merging and correction phases (e.g., handling rows with `"Acconciatore"`), only one inadmissible value remains: `"VIE"`. This value is not part of the valid labels defined in the `DESC_ABBREVIATA` column of the **tipovia** dataframe.

To resolve this, the `"VIE"` value must be corrected or mapped to a valid designation from the **tipovia** dataframe.

In [25]:
new_SERVICES[new_SERVICES["t_via"] == "VIE"]

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
721,Acconciatore,VIE DELLA ROBBIA LUCA N. 8 ; (z.d. 8),VIE,DELLA ROBBIA LUCA,8,6349.0,8,NaN,33.0
2891,Parrucchiere per Uomo;Tipo A - Estetica Manuale;Tipo B - Centro di Abbronzatura,VIE DELLA ROBBIA LUCA N. 8 ; (z.d. 8),VIE,DELLA ROBBIA LUCA,8,6349.0,8,NaN,NaN


In [26]:
new_SERVICES.loc[new_SERVICES["t_via"] == "VIE", "t_via"] = "VIA"

Once this adjustment is made, we ensure that all `t_via` values match the `DESC_ABBREVIATA` column of **tipovia** and can be directly mapped to their corresponding `DESC_ESTESA` values.

In [27]:
# Create a mapping dictionary from tipovia
t_via_mapping = dict(zip(tipovia["DESC_ABBREVIATA"].str.lower(), tipovia["DESC_ESTESA"]))

# Replace the t_via values in new_SERVICES with the corresponding DESC_ESTESA values
new_SERVICES["t_via"] = new_SERVICES["t_via"].str.lower().map(t_via_mapping)

# Check the result
new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
0,,LGO DEI GELSOMINI N. 10 (z.d. 6),Largo,DEI GELSOMINI,10,5394.0,6,NaN,55.0
1,Centro Massaggi,PZA FIDIA N. 3 (z.d. 9),Piazza,FIDIA,3,1144.0,9,2.0,28.0
2,Centro Benessere,VIA ADIGE N. 10 (z.d. 5),Via,ADIGE,10,4216.0,5,2.0,27.0
3,Truccatore,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),Via,BARACCHINI FLAVIO,9,356.0,1,NaN,NaN
4,,VIA BERGAMO N. 12 (z.d. 4),Via,BERGAMO,12,3189.0,4,NaN,50.0


### COD_VIA: Checking Format
This check ensures that the values in the `cod_via` column, which are stored as floats, are whole numbers (i.e., integers), since the identifiers related to street or location codes should always be integers.

In [28]:
# Check if any value in 'cod_via' is not an integer (checking for float values that are whole numbers)
non_integer_values = new_SERVICES['cod_via'].apply(lambda x: not x.is_integer() if isinstance(x, float) else False)

if non_integer_values.any():
    print("There are values in 'cod_via' that are not integers.")
else:
    print("All values in 'cod_via' are integers (float equivalents of integers).")

new_SERVICES.cod_via = new_SERVICES.cod_via.astype(float).astype(int).astype(str)

All values in 'cod_via' are integers (float equivalents of integers).


### CIV: Checking Format
This code checks the `civ` column to ensure that all values follow the expected format: **1-3 digit, optionally followed by an uppercase letter (e.g., 12A), a slash "/" with a single digit (e.g., 93/1), or a slash "/" and another series of digits and letters (e.g., 15/101).

In [29]:
# Define a function to check the format
def check_civ_format(value):
    # Regex pattern to match 1-3 digits, optionally followed by a `/` and a single digit or an uppercase letter
    pattern = r'^\d{1,3}(?:/\d{1,3}[A-Z]?)?(?:[A-Z]+)?(?:[A-Z]+P\d{2})?(?:[A-Z]+N\d{2})?(?:[A-Z]+C\d{2})?(?:[A-Z]+G\d{2})?$'
    return bool(re.match(pattern, str(value)))

# Apply the check to the 'civ' column and filter invalid values
invalid_civ_values = new_SERVICES['civ'][new_SERVICES['civ'].apply(lambda x: not check_civ_format(x) if pd.notnull(x) else False)]

if not invalid_civ_values.empty:
    print("There are invalid values in 'civ' column.\n")
    
if not invalid_civ_values.empty:
    for idx, value in invalid_civ_values.items():
        print(f"The row with index {idx} contains an invalid value: {value}.")
else:
    print("All values in 'civ' follow the correct format.")

There are invalid values in 'civ' column.

The row with index 143 contains an invalid value: 40945.


We handle invalid values in the `civ` column by replacing them with `NaN`, ensuring that only correctly formatted entries remain for further processing.

In [30]:
new_SERVICES[143:].head(1)

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
143,Acconciatore,VIA PAVIA N. 6/2 (z.d. 5),Via,PAVIA,40945,5262,5,NaN,NaN


In [31]:
# Replace invalid 'civ' values with NaN
new_SERVICES['civ'] = new_SERVICES['civ'].apply(lambda x: np.nan if not check_civ_format(x) else x)
new_SERVICES[143:].head(1)

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
143,Acconciatore,VIA PAVIA N. 6/2 (z.d. 5),Via,PAVIA,NaN,5262,5,NaN,NaN


In [32]:
print("The number of different values for column 'main' is {}.".format(new_SERVICES.civ.unique().shape[0]))

The number of different values for column 'main' is 234.


### VIA: Removal of Redundant Information and Validation

When processing records, some entries contain redundant information in the `via` column that matches part of the `t_via` column.
To ensure consistency and follow a specific order, we remove the redundant portion.

**Example**

Given the following data:

| t_es          | ubic                      | t_via | via      | civ | cod_via | zd | sup_alt | sup_lav |
|---------------|---------------------------|-------|----------|-----|---------|----|---------|---------|
| Acconciatore  | CSO COMO N. 15 (z.d. 9)  | CSO   | CSO COMO     | 15  | 1111.0  | 9  | NaN     | 195.0   |


The updated record becomes:

| t_es          | ubic                      | t_via | via      | civ | cod_via | zd | sup_alt | sup_lav |
|---------------|---------------------------|-------|----------|-----|---------|----|---------|---------|
| Acconciatore  | CSO COMO N. 15 (z.d. 9)  | CSO   | COMO     | 15  | 1111.0  | 9  | NaN     | 195.0   |


In this transformation:
- The `via` column value `CSO COMO` had the redundant prefix `CSO` matching the value in the `t_via` column.
- The prefix was removed, leaving only the unique portion: `COMO`.

In [33]:
new_SERVICES.via = new_SERVICES.via.fillna("").astype(str)
new_SERVICES.t_via = new_SERVICES.t_via.fillna("").astype(str) 

# Function to remove the redundant prefix
def clean_via(row):
    if row["via"].startswith(row["t_via"] + " ") and row["t_via"] != "":  # Check if via starts with t_via
        return row["via"].replace(row["t_via"] + " ", "")  # Remove t_via + space
    return row["via"]  # Return original if no match

# Apply the function to the 'via' column
new_SERVICES["via"] = new_SERVICES.apply(clean_via, axis=1)

new_SERVICES[new_SERVICES.via == "COMO"].head(1)

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
1140,Parrucchiere per Uomo,CSO COMO N. 11 ; (z.d. 9),Corso,COMO,11,1111,9,NaN,23.0


Now we validate the `via` column in the dataset by comparing it against the descriptive fields (`DESCRITTIVO` and `DENOMINAZIONE`) from the **viario** dataframe.<br> We first normalize the text by converting it to lowercase, removing unnecessary spaces and parentheses, and sorting the words alphabetically. Then, we calculate similarity scores between the `via` column and the two descriptive fields, using the highest score to flag rows for review if they fall below a **75%** similarity threshold. 

In [34]:
# Function to normalize the names by removing extra spaces, converting to lowercase, and sorting words
def normalize_string(s):

    #to remove specific pattern contained in the data
    s = re.sub(r"sant'\s*", "sant'", s.lower())  # Fix spaces around 'sant'
    s = re.sub(r"dell'\s*", "dell'", s.lower())  # Fix spaces around 'dell
    s = re.sub(r"[()]", "", s) # Remove the parenthesis
    
    # Split the string into words
    words = s.strip().split()
    
    # Sort the words alphabetically
    words.sort()
    
    # Join the words back together and convert to lowercase
    return " ".join(words)

# Function to compute the edit distance between two strings (after normalization)
def calculate_similarity(str1, str2):
    return fuzz.ratio(str1, str2)

# Merge new_SERVICES with viario on 'cod_via' and 'CODICE_VIA' to bring in 'DESCRITTIVO'
merged_df = new_SERVICES.merge(viario[['CODICE_VIA', 'DESCRITTIVO', 'DENOMINAZIONE']], left_on='cod_via', right_on='CODICE_VIA', how='left')

# Normalize 'via' and 'DESCRITTIVO' columns
merged_df['normalized_via'] = merged_df['via'].apply(normalize_string)
merged_df['normalized_descrittivo'] = merged_df['DESCRITTIVO'].apply(normalize_string)
merged_df['normalized_denominazione'] = merged_df['DENOMINAZIONE'].apply(normalize_string)

# Calculate the similarity between normalized values using edit distance
merged_df['similarity'] = merged_df.apply(lambda row: max(calculate_similarity(row['normalized_via'], row['normalized_descrittivo']), calculate_similarity(row['normalized_via'], row['normalized_denominazione'])), axis=1)

# Print rows where the similarity is below a certain threshold (e.g., 80)
threshold = 75
non_matching_rows = merged_df[merged_df['similarity'] < threshold]

# Show the rows with low similarity
print(f"Rows where normalized 'via' and 'DESCRITTIVO'/'DENOMINAZIONE' are not similar enough (below {threshold}% similarity):")
non_matching_rows[['cod_via', 'via', 'DENOMINAZIONE', 'DESCRITTIVO', 'similarity']].sort_values(by="similarity", ascending=True)

Rows where normalized 'via' and 'DESCRITTIVO'/'DENOMINAZIONE' are not similar enough (below 75% similarity):


,cod_via,via,DENOMINAZIONE,DESCRITTIVO,similarity
3087,612,DANTE ALIGHIERI,DANTE,DANTE,50.000000
2558,1524,GRAZIANO IMPERATORE,GRAZIANO,GRAZIANO,59.259259
3322,1524,GRAZIANO IMPERATORE,GRAZIANO,GRAZIANO,59.259259
2308,6380,TIZIANO VECELLIO,TIZIANO,TIZIANO,60.869565
3453,6380,TIZIANO VECELLIO,TIZIANO,TIZIANO,60.869565
363,7158,COLONNA MARCO ANTONIO,COLONNA MARCANTONIO,MARCANTONIO COLONNA,65.000000
2868,7158,COLONNA MARCO ANTONIO,COLONNA MARCANTONIO,MARCANTONIO COLONNA,65.000000
3612,7158,COLONNA MARCO ANTONIO,COLONNA MARCANTONIO,MARCANTONIO COLONNA,65.000000
32,1111,CSO COMO,COMO,COMO,66.666667
1572,4144,DON BOSCO,BOSCO DON GIOVANNI,DON GIOVANNI BOSCO,66.666667


Since these values with similarity lower than 75% match the respective addresses semantically, the `via` column can now be standardized to match the format of the `viario.DESCRITTIVO` column.

In [35]:
# Merge new_SERVICES with viario on the matching cod_via and CODICE_VIA
new_SERVICES = new_SERVICES.merge(viario[['CODICE_VIA', 'DESCRITTIVO']], 
                                  left_on='cod_via', right_on='CODICE_VIA', 
                                  how='left')

# Update the via column with the corresponding DESCRITTIVO value
new_SERVICES['via'] = new_SERVICES['DESCRITTIVO']

# Drop the extra columns (if needed)
new_SERVICES.drop(columns=['CODICE_VIA', 'DESCRITTIVO'], inplace=True)
new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
0,,LGO DEI GELSOMINI N. 10 (z.d. 6),Largo,DEI GELSOMINI,10,5394,6,NaN,55.0
1,Centro Massaggi,PZA FIDIA N. 3 (z.d. 9),Piazza,FIDIA,3,1144,9,2.0,28.0
2,Centro Benessere,VIA ADIGE N. 10 (z.d. 5),Via,ADIGE,10,4216,5,2.0,27.0
3,Truccatore,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),Via,FLAVIO BARACCHINI,9,356,1,NaN,NaN
4,,VIA BERGAMO N. 12 (z.d. 4),Via,BERGAMO,12,3189,4,NaN,50.0


### UBIC: Structuring and Format Changing
We process the `ubic` field to extract and organize its contents into distinct components: `ubic_t_via` (street type), `ubic_via` (street name), `ubic_civ` (street number), `ubic_zd` (zone description), and `note` (additional information). This step allows us to compare these extracted values with the corresponding columns that share similar semantic meanings (`t_via`, `via`, `civ`, and `zd`). 

Once the comparison is complete, we remove the `ubic` field and its derived columns to eliminate redundancy. 

This approach ensures a cleaner and more structured dataset for further analysis.

**Example**

Given the following data:

| t_es  | ubic | t_via | via | civ | cod_via | zd | sup_alt | sup_lav | 
|-------|------|-------|-----|-----|---------|----|---------|---------|
| Tipo A - Estetica Manuale;Tipo C - Trattamenti Estetici Dimagrimento | ALZ NAVIGLIO PAVESE N. 8 int.attivita' di parrucchiere; (z.d. 6) | ALZ | NAVIGLIO PAVESE | 8 | 5161.0 | 6 | NaN | NaN |

The updated record becomes:

| t_es  | t_via | via | civ | cod_via | zd | sup_alt | sup_lav | ubic_t_via | ubic_via | ubic_civ | ubic_zd | note |
|-------|-------|-----|-----|---------|----|---------|---------|------------|----------|----------|--------|------|
| Tipo A - Estetica Manuale;Tipo C - Trattamenti Estetici Dimagrimento  | ALZ | NAVIGLIO PAVESE | 8 | 5161.0 | 6 | NaN | NaN | ALZ | NAVIGLIO PAVESE | 8 | 6 | int.attivita' di parrucchiere |

**Explanation**:

In this transformation:
- The `via` column value `NAVIGLIO PAVESE` had the redundant prefix `ALZ` matching the value in the `t_via` column.
- The prefix `ALZ` is removed from the `via` column, leaving the unique portion: `NAVIGLIO PAVESE`.
- The `ubic` field remains the same in structure but is used to extract the relevant information, including the street name, number, and zone description, which are now placed in separate columns (`ubic_via`, `ubic_civ`, `ubic_zd`), while any additional information (e.g., `int.attivita' di parrucchiere`) is placed in the `note` column.

In [36]:
def split_ubicazione(ubicazione):
    # Initialize variables
    tipo_via = np.nan
    via = np.nan
    civico = np.nan
    zd = np.nan
    note = np.nan
    
    ubicazione = ubicazione.replace("Ã¸", "A")   # Handle lone 'Ã'
    ubicazione = ubicazione.replace(" n."," N.")
    
    # Split the string into parts
    parts = ubicazione.split(" ")

    if parts[0].lower() == "vie":
        parts[0] = "via"
    
    # Extract "Tipo Via" (first word if valid)
    if parts[0].lower() in tipovia.DESC_ABBREVIATA.str.lower().unique():
        tipo_via = parts[0]
        parts = parts[1:]  # Remove 'Tipo Via' from further processing

    # Find "Civico" (number after "N." or "num.")
    # pattern = r'^\d{1,3}(?:/\d{1,3}[A-Z]?)?(?:[A-Z]+)?(?:[A-Z]+P\d{2})?(?:[A-Z]+N\d{2})?(?:[A-Z]+C\d{2})?(?:[A-Z]+G\d{2})?$'
    pattern = r'\b(?:N\.|num\.)\s*([\w/]+)'
    match = re.search(pattern, ubicazione, re.IGNORECASE)
    if match:
        civico = match.group(1).strip(";")
        parts = re.sub(pattern, '', ubicazione).split()

    # Extract "ZD" (zone description)
    match_zd = re.search(r'\(z\.d\.\s*(\d+)\)', ubicazione, re.IGNORECASE)
    if match_zd:
        zd = match_zd.group(1)
        parts = re.sub(r'\(z\.d\.\s*\d+\)', '', " ".join(parts)).split()

    # To put together strings of the note like '1A' and 'piano' 
    # Check if there are at least three words to avoid IndexError
    if len(parts) > 3:
        # Define a regular expression to match "number + letter" (e.g., "1A")
        pattern = r"^\d+[A-Za-z]$"
        
        # Check if either parts[-2] or parts[-1] matches the pattern
        if re.match(pattern, parts[-2]) or re.match(pattern, parts[-1]):
            # Concatenate third-to-last and second-to-last parts
            parts[-2] = parts[-2] + " " + parts[-1]
            del parts[-1]  # Remove the last word after merging

    # Identify and extract the "Via" (remaining uppercase parts)
    via_parts = []
    note_parts = []
    for part in parts:
        if part.isupper():  # Uppercase letters are part of "Via"
            via_parts.append(part)
        else:  # Lowercase letters are considered "Note"
            note_parts.append(part)

    # Construct the "Via" without the "Tipo Via"
    via_parts = [part for part in via_parts if part != tipo_via]  # Remove 'tipo_via' from 'via_parts'
    via = " ".join(via_parts).strip()
    via =  via.replace(";", "")
    note = " ".join(note_parts).strip(";") if note_parts else np.nan
    if note == "":
        note = np.nan
    
    # Remove leading zeros unless the value contains alphabetic characters
    if civico and civico.isdigit():
        civico = str(int(civico))  # Remove leading zeros by converting to an integer
    elif civico:  # Preserve alphanumeric strings like '012B'
        civico = civico.lstrip("0").upper()  # Strip leading zeros but keep characters intact

    return tipo_via, via, civico, zd, note

# Apply the function to create new columns
new_SERVICES[['ubic_t_via', 'ubic_via', 'ubic_civ', 'ubic_zd', 'note']] = new_SERVICES['ubic'].apply(lambda x: pd.Series(split_ubicazione(x)))
new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,ubic_t_via,ubic_via,ubic_civ,ubic_zd,note
0,,LGO DEI GELSOMINI N. 10 (z.d. 6),Largo,DEI GELSOMINI,10,5394,6,NaN,55.0,LGO,DEI GELSOMINI,10,6,NaN
1,Centro Massaggi,PZA FIDIA N. 3 (z.d. 9),Piazza,FIDIA,3,1144,9,2.0,28.0,PZA,FIDIA,3,9,NaN
2,Centro Benessere,VIA ADIGE N. 10 (z.d. 5),Via,ADIGE,10,4216,5,2.0,27.0,VIA,ADIGE,10,5,NaN
3,Truccatore,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),Via,FLAVIO BARACCHINI,9,356,1,NaN,NaN,VIA,BARACCHINI FLAVIO,9,1,NaN
4,,VIA BERGAMO N. 12 (z.d. 4),Via,BERGAMO,12,3189,4,NaN,50.0,VIA,BERGAMO,12,4,NaN


In [37]:
new_SERVICES = new_SERVICES.drop("ubic", axis=1)
new_SERVICES.head()

,t_es,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,ubic_t_via,ubic_via,ubic_civ,ubic_zd,note
0,,Largo,DEI GELSOMINI,10,5394,6,NaN,55.0,LGO,DEI GELSOMINI,10,6,NaN
1,Centro Massaggi,Piazza,FIDIA,3,1144,9,2.0,28.0,PZA,FIDIA,3,9,NaN
2,Centro Benessere,Via,ADIGE,10,4216,5,2.0,27.0,VIA,ADIGE,10,5,NaN
3,Truccatore,Via,FLAVIO BARACCHINI,9,356,1,NaN,NaN,VIA,BARACCHINI FLAVIO,9,1,NaN
4,,Via,BERGAMO,12,3189,4,NaN,50.0,VIA,BERGAMO,12,4,NaN


Now we standardize the `ubic_t_via` column with the values from the **tipovia** dataframe. Since we have already ensured that all values in `ubic_t_via` match the corresponding abbreviations from the `DESC_ABBREVIATA` column in **tipovia**, there is no need for additional corrections or formatting. We directly map the `ubic_t_via` values to their corresponding `DESC_ESTESA` values from **tipovia** as `desc_attesa`.

In [38]:
# Create a mapping dictionary from tipovia for ubic_t_via
ubic_t_via_mapping = dict(zip(tipovia["DESC_ABBREVIATA"].str.lower(), tipovia["DESC_ESTESA"]))

# Replace the ubic_t_via values in new_SERVICES with the corresponding DESC_ESTESA values
new_SERVICES["ubic_t_via"] = new_SERVICES["ubic_t_via"].str.lower().map(ubic_t_via_mapping)

# Check the result
new_SERVICES.head()

,t_es,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,ubic_t_via,ubic_via,ubic_civ,ubic_zd,note
0,,Largo,DEI GELSOMINI,10,5394,6,NaN,55.0,Largo,DEI GELSOMINI,10,6,NaN
1,Centro Massaggi,Piazza,FIDIA,3,1144,9,2.0,28.0,Piazza,FIDIA,3,9,NaN
2,Centro Benessere,Via,ADIGE,10,4216,5,2.0,27.0,Via,ADIGE,10,5,NaN
3,Truccatore,Via,FLAVIO BARACCHINI,9,356,1,NaN,NaN,Via,BARACCHINI FLAVIO,9,1,NaN
4,,Via,BERGAMO,12,3189,4,NaN,50.0,Via,BERGAMO,12,4,NaN


Next, we validate the `ubic_via` column in the dataset by comparing it to the descriptive fields (`DESCRITTIVO` and `DENOMINAZIONE`) from the `viario` dataframe, following a similar process as for the `via` column. <br> First, we normalize the text by converting it to lowercase, removing extraneous spaces and parentheses (while preserving their contents), and sorting the words alphabetically to address potential inconsistencies. Then, we calculate similarity scores based on **Edit Distanca** between the `ubic_via` column and both `DESCRITTIVO` and `DENOMINAZIONE`.

The highest similarity score is selected for each row, and rows with a similarity score below 75% are flagged for further review.

In [39]:
# Function to normalize the names
def normalize_string(s):
    s = re.sub(r"sant'\s*", "sant'", s.lower())  # Fix spaces around 'sant'
    s = re.sub(r"dell'\s*", "dell'", s.lower())  # Fix spaces around 'dell'
    s = re.sub(r"[()]", "", s)  # Remove the parentheses
    
    # Split the string into words, sort them, and rejoin
    words = s.strip().split()
    words.sort()
    return " ".join(words)

# Pre-normalize the viario DataFrame
viario['normalized_descrittivo'] = viario['DESCRITTIVO'].apply(normalize_string)
viario['normalized_denominazione'] = viario['DENOMINAZIONE'].apply(normalize_string)

# Function to find the best match using pre-normalized viario
def find_best_match(ubic_via):

    measure = "EDIT"
    normalized_ubic_via = normalize_string(ubic_via)
    
    # Compute similarity scores for all rows in viario
    viario['similarity_descrittivo'] = viario['normalized_descrittivo'].apply(
        lambda x: fuzz.ratio(normalized_ubic_via, x))
    viario['similarity_denominazione'] = viario['normalized_denominazione'].apply(
        lambda x: fuzz.ratio(normalized_ubic_via, x))
    
    # Find the best match
    viario['similarity'] = viario[['similarity_descrittivo', 'similarity_denominazione']].max(axis=1)
    best_match = viario.loc[viario['similarity'].idxmax()]
    
    return (measure, best_match['similarity'],
            ubic_via, best_match['DENOMINAZIONE'],best_match['DESCRITTIVO'],
            normalized_ubic_via, best_match['normalized_denominazione'], best_match['normalized_descrittivo'],
            )

# Process all rows in new_SERVICES
results = []

for ubic_via in tqdm(new_SERVICES['ubic_via'].unique()):
    ubic_via_data = find_best_match(ubic_via)
    results.append(ubic_via_data)

# Convert results to DataFrame for analysis
columns = ['measure','similarity',
            'ubic_via', 'best_denominazione', 'best_descrittivo', 'normalized_ubic_via',
            'best_normalized_denominazione', 'best_normalized_descrittivo', 
            ]
check_matching_df = pd.DataFrame(results, columns=columns)

# Show the rows with low similarity
print(f"Rows where normalized 'ubic_via' and 'DESCRITTIVO'/'DENOMINAZIONE' are not similar enough (below {threshold}% Edit Distance similarity):")
check_matching_df.sort_values(by="similarity", ascending=True)

100%|██████████| 1381/1381 [00:11<00:00, 123.40it/s]

Rows where normalized 'ubic_via' and 'DESCRITTIVO'/'DENOMINAZIONE' are not similar enough (below 75% Edit Distance similarity):


,measure,similarity,ubic_via,best_denominazione,best_descrittivo,normalized_ubic_via,best_normalized_denominazione,best_normalized_descrittivo
183,EDIT,0.000000,,DUOMO (DEL),DEL DUOMO,,del duomo,del duomo
1172,EDIT,63.157895,GRAZIANO IMPERATORE,VIMERCATI OTTAVIANO,OTTAVIANO VIMERCATI,graziano imperatore,ottaviano vimercati,ottaviano vimercati
1103,EDIT,64.705882,TIZIANO VECELLIO,AMMIANO MARCELLINO,MARCELLINO AMMIANO,tiziano vecellio,ammiano marcellino,ammiano marcellino
1265,EDIT,68.750000,DANTE ALIGHIERI,CHIASSERINI DANTE,DANTE CHIASSERINI,alighieri dante,chiasserini dante,chiasserini dante
292,EDIT,71.428571,COLONNA MARCO ANTONIO,CARLONE CARLO ANTONIO,CARLO ANTONIO CARLONE,antonio colonna marco,antonio carlo carlone,antonio carlo carlone
...,...,...,...,...,...,...,...,...
461,EDIT,100.000000,VENINI GIULIO E CORRADO,VENINI GIULIO E CORRADO,GIULIO E CORRADO VENINI,corrado e giulio venini,corrado e giulio venini,corrado e giulio venini
460,EDIT,100.000000,VEGLIA,VEGLIA,VEGLIA,veglia,veglia,veglia
459,EDIT,100.000000,VARESE,VARESE,VARESE,varese,varese,varese
467,EDIT,100.000000,VIMINALE,VIMINALE,VIMINALE,viminale,viminale,viminale


Some rows fall below the minimum similarity threshold of 75% when evaluated using edit distance. To gain further insight into these records, we reanalyze them using the **Jaccard** similarity measure.

In [40]:
# Function to compute Jaccard similarity
def compute_jaccard(s1, s2):
    vectorizer = CountVectorizer().fit_transform([s1, s2])
    vectors = vectorizer.toarray()
    return 1 - jaccard(vectors[0], vectors[1])

# Function to compute Jaccard similarity for all rows in viario and update non_matching_df
def compute_jaccard_for_all_rows():
    for index, row in tqdm(check_matching_df[check_matching_df.similarity < 77].iterrows(), total=len(check_matching_df[check_matching_df.similarity < 77])):
        normalized_ubic_via = row['normalized_ubic_via']
        
        # Compute Jaccard similarity for 'descrittivo' and 'denominazione' for all rows in viario
        viario['jaccard_descrittivo'] = viario['normalized_descrittivo'].apply(
            lambda x: compute_jaccard(normalized_ubic_via, x))
        viario['jaccard_denominazione'] = viario['normalized_denominazione'].apply(
            lambda x: compute_jaccard(normalized_ubic_via, x))
        
        # Take the maximum Jaccard similarity (between 'descrittivo' and 'denominazione')
        viario['jaccard_similarity'] = viario[['jaccard_descrittivo', 'jaccard_denominazione']].max(axis=1)
        
        # Find the best match based on the highest Jaccard similarity
        best_match = viario.loc[viario['jaccard_similarity'].idxmax()]
        
        # Update the non_matching_df with the best match and Jaccard similarity
        check_matching_df.at[index, 'best_normalized_descrittivo'] = best_match['normalized_descrittivo']
        check_matching_df.at[index, 'best_normalized_denominazione'] = best_match['normalized_denominazione']
        check_matching_df.at[index, 'best_descrittivo'] = best_match['DESCRITTIVO']
        check_matching_df.at[index, 'best_denominazione'] = best_match['DENOMINAZIONE']
        check_matching_df.at[index, 'measure'] = "JACCARD"
        check_matching_df.at[index, 'similarity'] = best_match['jaccard_similarity'] * 100

# Run Jaccard computation for all rows in viario and update non_matching_df
compute_jaccard_for_all_rows()

# Show the updated non_matching_df with Jaccard similarity values
print(f"Updated non_matching_df with Jaccard similarity:")
check_matching_df.sort_values(by="similarity")

100%|██████████| 7/7 [00:30<00:00,  4.39s/it]

Updated non_matching_df with Jaccard similarity:


,measure,similarity,ubic_via,best_denominazione,best_descrittivo,normalized_ubic_via,best_normalized_denominazione,best_normalized_descrittivo
183,JACCARD,0.0,,DUOMO (DEL),DEL DUOMO,,del duomo,del duomo
292,JACCARD,50.0,COLONNA MARCO ANTONIO,BRAGADINO MARCO ANTONIO,MARCO ANTONIO BRAGADINO,antonio colonna marco,antonio bragadino marco,antonio bragadino marco
1172,JACCARD,50.0,GRAZIANO IMPERATORE,GRAZIANO,GRAZIANO,graziano imperatore,graziano,graziano
1265,JACCARD,50.0,DANTE ALIGHIERI,DANTE,DANTE,alighieri dante,dante,dante
1103,JACCARD,50.0,TIZIANO VECELLIO,TIZIANO,TIZIANO,tiziano vecellio,tiziano,tiziano
...,...,...,...,...,...,...,...,...
461,EDIT,100.0,VENINI GIULIO E CORRADO,VENINI GIULIO E CORRADO,GIULIO E CORRADO VENINI,corrado e giulio venini,corrado e giulio venini,corrado e giulio venini
460,EDIT,100.0,VEGLIA,VEGLIA,VEGLIA,veglia,veglia,veglia
459,EDIT,100.0,VARESE,VARESE,VARESE,varese,varese,varese
467,EDIT,100.0,VIMINALE,VIMINALE,VIMINALE,viminale,viminale,viminale


With the exception of two rows, which could not be recognized due to missing values in the `ubic_via` column, most of the rows now exhibit a similarity of 50% or greater, indicating that their addresses match semantically. As a result, the `ubic_via` column can now be standardized to align with the format of the `viario.DESCRITTIVO` column.

In [41]:
# Step 1: Merge new_SERVICES with check_matching_df on 'ubic_via'
merged_df = new_SERVICES.merge(check_matching_df[['ubic_via', 'best_descrittivo', 'similarity']], 
                               on='ubic_via', how='left')

# Step 2: Filter rows where similarity >= 50
filtered_df = merged_df[merged_df['similarity'] >= 50]

# Step 3: Update the 'ubic_via' column in new_SERVICES with 'DENOMINAZIONE' from check_matching_df where similarity >= 50
for index, row in filtered_df.iterrows():
    new_SERVICES.loc[new_SERVICES['ubic_via'] == row['ubic_via'], 'ubic_via'] = row['best_descrittivo']

new_SERVICES.head()

,t_es,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,ubic_t_via,ubic_via,ubic_civ,ubic_zd,note
0,,Largo,DEI GELSOMINI,10,5394,6,NaN,55.0,Largo,DEI GELSOMINI,10,6,NaN
1,Centro Massaggi,Piazza,FIDIA,3,1144,9,2.0,28.0,Piazza,FIDIA,3,9,NaN
2,Centro Benessere,Via,ADIGE,10,4216,5,2.0,27.0,Via,ADIGE,10,5,NaN
3,Truccatore,Via,FLAVIO BARACCHINI,9,356,1,NaN,NaN,Via,FLAVIO BARACCHINI,9,1,NaN
4,,Via,BERGAMO,12,3189,4,NaN,50.0,Via,BERGAMO,12,4,NaN


## Error Detection and Correction
### UBIC-VIA: Solve Mismatches

The goal is to resolve mismatches between the `ubic` columns (`ubic_t_via`, `ubic_via`, `ubic_civ`, `ubic_zd`) and the corresponding main columns (`t_via`, `via`, `civ`, `zd`). Mismatches are identified by comparing values in these columns. 

Once detected, corrections are made by updating the values to ensure consistency, improving the dataset's quality and accuracy for analysis.

In [42]:
# Initialize dictionary to store mismatches
mismatch_rows = {
    "note empty": [],
    "t_via not match": [],
    "civ not match": [],
    "via not match": [],
    "zd not match": [],
}

# Function to check conditions and log mismatched rows
def check_and_log_mismatches(new_SERVICES):
    # Create masks for each condition (True if mismatch, False if match)
    note_not_empty_condition = ~new_SERVICES['note'].isna()
    t_via_not_match_condition = new_SERVICES["t_via"] != new_SERVICES["ubic_t_via"]
    civ_not_match_condition = new_SERVICES["civ"] != new_SERVICES["ubic_civ"]
    via_not_match_condition = new_SERVICES["via"] != new_SERVICES["ubic_via"]
    zd_not_match_condition = new_SERVICES["zd"] != new_SERVICES["ubic_zd"]
    
    # Apply the conditions to filter rows and store them in the corresponding lists
    mismatch_rows["note empty"] = new_SERVICES[note_not_empty_condition].to_dict(orient='records')
    mismatch_rows["t_via not match"] = new_SERVICES[t_via_not_match_condition].to_dict(orient='records')
    mismatch_rows["civ not match"] = new_SERVICES[civ_not_match_condition].to_dict(orient='records')
    mismatch_rows["via not match"] = new_SERVICES[via_not_match_condition].to_dict(orient='records')
    mismatch_rows["zd not match"] = new_SERVICES[zd_not_match_condition].to_dict(orient='records')
    
    return new_SERVICES

# Apply the function to check and log mismatches
new_SERVICES = check_and_log_mismatches(new_SERVICES)

# Print summary of mismatches with aligned output
print("Summary of mismatches:\n")
# Print a header row with column names
print(f"{'Mismatch Type':<20} {'Number of Mismatches':<20}")
print("="*40)  # Separator for clarity

# Iterate over mismatch_rows and print each mismatch type and its count
for key, value in mismatch_rows.items():
    print(f"{key:<20} \t\t{len(value):<20}")

Summary of mismatches:

Mismatch Type        Number of Mismatches
note empty           		396                 
t_via not match      		20                  
civ not match        		150                 
via not match        		76                  
zd not match         		17                  


Check if the full `ubic` and `via_full` addresses are included in the dataset of **civici**.

In [43]:
# Define the function to check full_via existence
def check_full_via_exists(row, civici_df):
    # Check if the necessary columns contain NaN values
    if pd.isna(row['ubic_t_via']) or pd.isna(row['ubic_via']) or pd.isna(row['t_via']) or pd.isna(row['via']):
        return [False, False]  # Return [False, False] if any critical columns have NaN values

    # Combine the address components if valid
    full_via_ubic = (row['ubic_t_via'].strip() + " " + row['ubic_via'].strip())
    full_via_via = (row['t_via'].strip() + " " + row['via'].strip())

    # Check if full_via_ubic exists in civici.DESCRITTIVO and match NUMEROCOMPLETO and MUNICIPIO
    match_ubic = civici_df[(civici_df['DESCRITTIVO'] == full_via_ubic) &
                           (civici_df['NUMEROCOMPLETO'] == row['ubic_civ']) &
                           (civici_df['MUNICIPIO'] == row['ubic_zd'].strip())]

    # Check if full_via_via exists in civici.DESCRITTIVO and match NUMEROCOMPLETO and MUNICIPIO
    match_via = civici_df[(civici_df['DESCRITTIVO'] == full_via_via) &
                          (civici_df['NUMEROCOMPLETO'] == row['civ']) &
                          (civici_df['MUNICIPIO'] == row['zd'].strip())]

    # Return flags as a list
    return [len(match_via) > 0, len(match_ubic) > 0]

# Enable tqdm for pandas
tqdm.pandas()

# Apply the check function to each row in your dataset
new_SERVICES[['full_via_flag', 'ubic_flag']] = new_SERVICES.progress_apply(
    lambda row: pd.Series(check_full_via_exists(row, civici)), axis=1
)

# Show the result
new_SERVICES.head()

100%|██████████| 3908/3908 [02:02<00:00, 31.97it/s]


,t_es,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,ubic_t_via,ubic_via,ubic_civ,ubic_zd,note,full_via_flag,ubic_flag
0,,Largo,DEI GELSOMINI,10,5394,6,NaN,55.0,Largo,DEI GELSOMINI,10,6,NaN,True,True
1,Centro Massaggi,Piazza,FIDIA,3,1144,9,2.0,28.0,Piazza,FIDIA,3,9,NaN,True,True
2,Centro Benessere,Via,ADIGE,10,4216,5,2.0,27.0,Via,ADIGE,10,5,NaN,True,True
3,Truccatore,Via,FLAVIO BARACCHINI,9,356,1,NaN,NaN,Via,FLAVIO BARACCHINI,9,1,NaN,True,True
4,,Via,BERGAMO,12,3189,4,NaN,50.0,Via,BERGAMO,12,4,NaN,True,True


In [44]:
valid_addresses = new_SERVICES.loc[new_SERVICES['ubic_flag'] == True]
print("The number of valid adresses from 'ubic' column composition is {}.\n".format(len(valid_addresses)))

valid_addresses = new_SERVICES.loc[new_SERVICES['full_via_flag'] == True]
print("The number of valid adresses from 't_via', 'via', 'civ' and 'zd'  composition is {}.".format(len(valid_addresses)))

The number of valid adresses from 'ubic' column composition is 3820.

The number of valid adresses from 't_via', 'via', 'civ' and 'zd'  composition is 3823.


We perform two filtering steps on the dataset to improve the reliability of our analysis. The reasoning behind these steps is as follows:

1. **First Filtering: Rows with Both Flags True but Mismatched Values**  
   We drop rows where both `ubic_flag` and `full_via_flag` are `True`, but the corresponding values (`via`, `ubic_via`, `zd`, `ubic_zd`, `civ`, `ubic_civ`) do not match.  
   - **Reason**: When both flags indicate correctness (`True`), the dataset gives conflicting information because the actual values do not align. In such cases, it is unclear which value is the true representation, leading to ambiguity. To avoid unreliable data, these rows are removed.

2. **Second Filtering: Rows with Both Flags False**  
   We also drop rows where both `ubic_flag` and `full_via_flag` are `False`.  
   - **Reason**: When both flags are `False`, neither source of information can be verified as correct. These rows provide no reliable information and are therefore discarded to maintain the dataset's integrity.

In [45]:
conditions_TT = (new_SERVICES.ubic_flag == True) & (new_SERVICES.full_via_flag == True) & \
            ((new_SERVICES.via != new_SERVICES.ubic_via) | (new_SERVICES.zd != new_SERVICES.ubic_zd) | \
            (new_SERVICES.civ != new_SERVICES.ubic_civ))

new_SERVICES = new_SERVICES[~conditions_TT]

conditions_FF = (new_SERVICES.ubic_flag == False) & (new_SERVICES.full_via_flag == False)

new_SERVICES = new_SERVICES[~conditions_FF]
print("The 'SERVICES' dataset has now {} records.".format(new_SERVICES.shape[0]))

The 'SERVICES' dataset has now 3845 records.


We perform a series of updates on the dataset to address rows where `full_via_flag` is `False` but `ubic_flag` is `True`. The operations are as follows:

1. **Value Transfer and Substitution**  
   We transfer the values from `ubic_t_via`, `ubic_via`, and `ubic_civ` to the respective fields `t_via`, `via`, and `civ`. Additionally, we update the `cod_via` by finding a matching record in the `civici` dataset based on the concatenated address (`ubic_t_via` + `ubic_via`) and `ubic_civ`.
   - **Reason**: When `ubic_flag` is `True`, it indicates that the `ubic` dataset provides reliable address data. However, the `full_via_flag` being `False` means that the target fields (`t_via`, `via`, `civ`) need to be populated. We ensure consistency by transferring the values and updating `cod_via` from the `civici` dataset.

2. **Address Matching with `civici`**  
   We check if the concatenated address from `ubic_t_via` and `ubic_via` matches the records in `civici` and update `cod_via` accordingly.
   - **Reason**: This ensures the `cod_via` value aligns with the correct address from the `civici` dataset, maintaining data integrity across related fields.

In [46]:
# Identify rows where the condition is True
conditions_TF = (new_SERVICES.ubic_flag == True) & (new_SERVICES.full_via_flag == False)

# Loop through the rows and handle the transfer and substitution
for index, row in new_SERVICES[conditions_TF].iterrows():
    # Transfer values from ubic_t_via, ubic_via, ubic_civ to t_via, via, civ
    new_SERVICES.at[index, 't_via'] = row['ubic_t_via']
    new_SERVICES.at[index, 'via'] = row['ubic_via']
    new_SERVICES.at[index, 'civ'] = row['ubic_civ']
    
    # Ensure proper concatenation of strings before the comparison
    full_address = row['ubic_t_via'].strip() + " " + row['ubic_via'].strip()

    # Find matching cod_via in the civici dataframe
    matching_civici = civici[(civici['DESCRITTIVO'] == full_address) &
                             (civici['NUMEROCOMPLETO'] == row['ubic_civ'])]
    
    # Substitute the cod_via if a match is found
    if not matching_civici.empty:
        new_SERVICES.at[index, 'cod_via'] = matching_civici['CODICE_VIA'].iloc[0]

# Check the result after the updates
new_SERVICES[conditions_TF].head()

,t_es,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,ubic_t_via,ubic_via,ubic_civ,ubic_zd,note,full_via_flag,ubic_flag
143,Acconciatore,Via,PAVIA,6/2,5262,5,NaN,NaN,Via,PAVIA,6/2,5,NaN,False,True
285,Acconciatore,Via,APPENNINI,77H,7514,8,NaN,NaN,Via,APPENNINI,77H,8,NaN,False,True
307,Acconciatore,Via,UGO BETTI,161A,7517,8,6.0,23.0,Via,UGO BETTI,161A,8,NaN,False,True
316,Acconciatore,Via,PIETRO BOIFAVA,6C,5341,5,7.0,20.0,Via,PIETRO BOIFAVA,6C,5,NaN,False,True
327,Acconciatore,Via,PIETRO CALVI,3A,3083,4,5.0,193.0,Via,PIETRO CALVI,3A,4,p.t.acconciatore,False,True


We drop all the rows that we don't need anymore: like ubic derivatives and flags.

In [47]:
new_SERVICES = new_SERVICES.drop(["ubic_t_via", "ubic_via", "ubic_civ", "ubic_zd", "full_via_flag", "ubic_flag"], axis=1)
new_SERVICES.head()

,t_es,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,note
0,,Largo,DEI GELSOMINI,10,5394,6,NaN,55.0,NaN
1,Centro Massaggi,Piazza,FIDIA,3,1144,9,2.0,28.0,NaN
2,Centro Benessere,Via,ADIGE,10,4216,5,2.0,27.0,NaN
3,Truccatore,Via,FLAVIO BARACCHINI,9,356,1,NaN,NaN,NaN
4,,Via,BERGAMO,12,3189,4,NaN,50.0,NaN


## Data Deduplication 
### Detecting and handling non-exact duplicates
In this operation, we address potential data corruption in the original dataset, where records associated with the same address might have been split into multiple rows. This splitting likely occurred because the dataset exploded certain records across multiple lines, associating different labels in the `t_es` column with separate rows.

The goal of this process is to consolidate these rows into a single entry per address when they share overlapping labels in the `t_es` column.

##### Example

To illustrate the issue, consider the following rows from the dataset:

| Index | t_es                                | via   | street_name    | civ | cod_via | sup_alt | sup_lav | note |
|-------|-------------------------------------|-------|----------------|-----|---------|---------|---------|------|
| 31    | Acconciatore                       | Corso | BUENOS AIRES   | 23  | 2129    | 3       | NaN     | 68.0 |
| 798   | Tipo A - Estetica Manuale;Acconciatore | Corso | BUENOS AIRES   | 23  | 2129    | 3       | NaN     | 45.0 |
| 3171  | Tipo A - Estetica Manuale          | Corso | BUENOS AIRES   | 23  | 2129    | 3       | NaN     | 45.0 |

which transforms to:

| Index | t_es                                | via   | street_name    | civ | cod_via | sup_alt | sup_lav | note |
|-------|-------------------------------------|-------|----------------|-----|---------|---------|---------|------|
| 0     | Tipo A - Estetica Manuale;Acconciatore | Corso | BUENOS AIRES   | 23  | 2129    | 3       | NaN     | 68.0 |

In this example, the rows, which are consolidated the same address, are consolidated into a single row. The `t_es` column combines all unique labels into one string, resulting in `"Tipo A - Estetica Manuale;Acconciatore"`. For numeric fields like `sup_alt` and `sup_lav`, the maximum values across the rows are retained.

Redundant rows are then removed, leaving only the merged entry.

In [48]:
# Define categories and their specializations
categories_to_merge = {
    "Acconciatore": ["Acconciatore", "Parrucchiere per Uomo", "Parrucchiere per Donna"], 
    "Estetica": ["Tipo A - Estetica Manuale", "Tipo B - Centro di Abbronzatura", 
                 "Tipo C - Trattamenti Estetici Dimagrimento", "Tipo D - Estetica Apparati Elettromeccanici"]
}

# Function to check if any label in multi-labeled t_es matches specializations
def should_merge(row1, row2):
    # Convert t_es values to lists if they are multi-labeled (e.g., "Acconciatore, Parrucchiere Uomo")
    t_es1_labels = row1["t_es"].split(";")
    t_es2_labels = row2["t_es"].split(";")

    if t_es1_labels[0] == "" or t_es2_labels[0] == "":
        return True
        
    for category, specializations in categories_to_merge.items():
        # Check if any label in t_es1 or t_es2 belongs to the current category's specializations
        if any(label.strip() in specializations for label in t_es1_labels) and any(label.strip() in specializations for label in t_es2_labels):
            return True
    return False

def merge_rows(group):
    merged_rows = []
    i = 0  # We will iterate row by row
    
    while i < len(group):
        row1 = group.iloc[i]
        merged = False  # Flag to check if the row is merged
        
        # Check if the current row can be merged with the next one
        if i + 1 < len(group):
            row2 = group.iloc[i + 1]
            
            if should_merge(row1, row2):  # Check if we should merge the rows
                # Merge the two rows
                merged_row = row1.copy()  # Start with the first row
                merged_row["t_es"] = ";".join(set(row1["t_es"].split(";") + row2["t_es"].split(";")))  # Merge t_es values
                if merged_row["t_es"][0] == ";":
                    merged_row["t_es"] = merged_row["t_es"][1:]

                # Assuming merged_row["t_es"] is a string, split it into a list
                t_es_list = merged_row["t_es"].split(";") if isinstance(merged_row["t_es"], str) else merged_row["t_es"]
                
                # Check if "Acconciatore" is in the list and if any other term from the same category exists
                if "Acconciatore" in t_es_list:
                    if any(term in t_es_list for term in categories_to_merge["Acconciatore"] if term != "Acconciatore"):
                        t_es_list.remove("Acconciatore")  # Remove "Acconciatore" from the list if condition is met
                
                # Join the list back into a string, if necessary
                merged_row["t_es"] = ";".join(t_es_list)

                
                merged_row["sup_alt"] = max(row1["sup_alt"], row2["sup_alt"])  # Take the max value
                merged_row["sup_lav"] = max(row1["sup_lav"], row2["sup_lav"])  # Take the max value
                
                merged_rows.append(merged_row)
                i += 2  # Skip the next row since it's already merged
                merged = True
        
        # If no merge occurred, add the current row to the result
        if not merged:
            merged_rows.append(row1)
            i += 1  # Move to the next row
    
    return pd.DataFrame(merged_rows) 

# Apply the merge_rows function to groups with more than 1 row
new_SERVICES = new_SERVICES.groupby(["cod_via", "via", "civ"]).apply(lambda group: merge_rows(group)).reset_index(drop=True)
new_SERVICES.head()

,t_es,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,note
0,Tipo A - Estetica Manuale,Piazza,DEL DUOMO,17,1,1,NaN,74.0,NaN
1,Tipo A - Estetica Manuale;Tipo B - Centro di Abbronzatura,Corso,GIUSEPPE GARIBALDI,104,1010,1,NaN,NaN,NaN
2,Tipo A - Estetica Manuale;Parrucchiere per Donna,Corso,GIUSEPPE GARIBALDI,110,1010,1,NaN,88.0,NaN
3,Acconciatore,Corso,GIUSEPPE GARIBALDI,39,1010,1,NaN,54.0,NaN
4,Parrucchiere per Donna,Corso,GIUSEPPE GARIBALDI,46,1010,1,NaN,NaN,NaN


Furthermore, `"Acconciatore"` is treated as a broader category, while `"Parrucchiere per Uomo"` and `"Parrucchiere per Donna"` are considered specific specializations within that category. <br> When these labels appear together in the same record, we merge them but also remove `"Acconciatore"` if any of these specific specializations are present. This is done because we recognize that the specific specializations are more relevant in this context. This helps to refine the dataset by ensuring that the more specific services take precedence over the broader category.

Then, we move on by removing duplicate rows based on the columns `t_es`, `cod_via`, `via`, and `civ`, so each record is unique. Rows where `t_es` is empty are also removed to keep only relevant data. Finally, the `note` column is deleted because it is not needed for further analysis, making the dataset simpler and easier to work with.

In [49]:
new_SERVICES = new_SERVICES.drop_duplicates(subset=["t_es", "cod_via", "via", "civ"])
new_SERVICES = new_SERVICES[~(new_SERVICES.t_es == "")]
new_SERVICES = new_SERVICES.drop(columns=["note"])
print("The 'SERVICES' dataset has now {} records.".format(new_SERVICES.shape[0]))

The 'SERVICES' dataset has now 3523 records.


## Outlier Detection
### SUP_LAV: Detecting and Removing Outliers

This step fills missing values in the `sup_lav` column by replacing them with the median value of `sup_lav` for each `t_es` category. We then detect outliers in the `sup_lav` column using the Z-score method, identifying values that are more than 3 standard deviations away from the mean. This helps standardize the data by addressing missing values and flagging extreme entries that could distort analysis.

In [50]:
# Step 1.1: Calculate the median "sup_lav" for each "t_es"
medians = new_SERVICES.groupby('t_es')['sup_lav'].median()

# Step 1.2: Fill null values with the median of the corresponding "t_es"
new_SERVICES['sup_lav'] = new_SERVICES.apply(
    lambda row: medians[row['t_es']] if pd.isnull(row['sup_lav']) else row['sup_lav'],
    axis=1
)

# Step 2: Perform outlier detection using the Z-score method
mean_superficie = new_SERVICES['sup_lav'].mean()
std_superficie = new_SERVICES['sup_lav'].std()

# Calculate the Z-score
new_SERVICES['z_score'] = (new_SERVICES['sup_lav'] - mean_superficie) / std_superficie

# Identify outliers (Z-score > 3 or < -3)
outliers = new_SERVICES[(new_SERVICES['z_score'] > 3) | (new_SERVICES['z_score'] < -3)]

# Print the number of outliers
print("Outliers detected using Z-score for the 'sup_lav' column are {}".format(outliers.shape[0]))

Outliers detected using Z-score for the 'sup_lav' column are 53


Then, we remove the outliers identified in the previous step by filtering out rows where the Z-score is greater than 3 or less than -3. After removing the outliers, we drop the `z_score` column as it is no longer needed for further analysis. This ensures the dataset is free from extreme values that could skew results.

In [51]:
# Step 4: Remove outliers
new_SERVICES = new_SERVICES[(new_SERVICES['z_score'] <= 3) & (new_SERVICES['z_score'] >= -3)]

# Drop the Z-score column as it's no longer needed
new_SERVICES = new_SERVICES.drop(columns=['z_score'])

### SUP_ALT: Detecting and Removing Outliers

We apply the same process to the `sup_alt` column: first, we calculate the Z-scores to identify outliers, then filter out rows where the Z-score for `sup_alt` is greater than 3 or less than -3. Finally, we drop the `z_score` column, ensuring that the dataset is cleaned of extreme values in both the `sup_lav` and `sup_alt` columns.

In [52]:
# Step 1.1: Calculate the median "sup_alt" for each "t_es"
medians = new_SERVICES.groupby('t_es')['sup_alt'].median()

# Step 1.2: Fill null values with the median of the corresponding "t_es"
new_SERVICES['sup_alt'] = new_SERVICES.apply(
    lambda row: medians[row['t_es']] if pd.isnull(row['sup_alt']) else row['sup_alt'],
    axis=1
)

# Step 2: Perform outlier detection using the Z-score method
mean_superficie = new_SERVICES['sup_alt'].mean()
std_superficie = new_SERVICES['sup_alt'].std()

# Calculate the Z-score
new_SERVICES['z_score'] = (new_SERVICES['sup_alt'] - mean_superficie) / std_superficie

# Identify outliers (Z-score > 3 or < -3)
outliers = new_SERVICES[(new_SERVICES['z_score'] > 3) | (new_SERVICES['z_score'] < -3)]

print("Outliers detected using Z-score for the 'sup_' column are {}".format(outliers.shape[0]))

# Step 4: Remove outliers
new_SERVICES = new_SERVICES[(new_SERVICES['z_score'] <= 3) & (new_SERVICES['z_score'] >= -3)]

# Drop the Z-score column as it's no longer needed
new_SERVICES = new_SERVICES.drop(columns=['z_score'])

Outliers detected using Z-score for the 'sup_' column are 31


After applying these steps, including filling missing values with the median and removing outliers, the dataset will no longer contain any `NaN` values. All missing data in the `sup_lav` and `sup_alt` columns will have been addressed, ensuring a complete and clean dataset without any `NaN` entries.

In [53]:
new_SERVICES.isna().sum()

t_es       0
t_via      0
via        0
civ        0
cod_via    0
zd         0
sup_alt    0
sup_lav    0
dtype: int64

In [54]:
new_SERVICES.to_csv("cleaned-SERVICES.csv", index=False)